In [51]:
# 1.0 Feladat megfogalmazása:
#
# Szöveges mintákhoz tartozó értékek beazonosítása. A minták megtalálása a sok variáció miatt klasszikus módszerrel nehéz.

# 1.1 Lehetséges minták például:
#
# "az arteria carotis communisban ... % stenosist okozó echogen plaque."
# "az arteria carotis interna elején ... % stenosist okozó echogen plaque.",
# "carotis oszlásban (bifurcatioban) ... % stenosist okozó echogen plaque.",
# "a carotis externában ... % stenosist okozó echogen plaque.",
# "arteria vertebralisban az áramlási sebesség a norm. érték alsó határának ... %-a.",
# "arteria basilarisban az áramlási sebesség a norm. érték alsó határának ... %-a.

# 1.2 A ... -helyeken bármilyen érték álhat:
#
# Példák a lehetséges értékekre:
# 10, 20, 10-20, kb.10, kb.50 stb...

# 1.3 A minták időben lehet módosulta. A módosulás mértékről nincs információ:
#
# Példa a modósulásara: Az "echogen" szó felcserélődött a "meszes" szóra
# Például következő minták szemantikai jelentése azonos.
# "az arteria carotis communisban ... % stenosist okozó echogen plaque."
# "az arteria carotis communisban ... % stenosist okozó meszes plaque."
# "az arteria carotis communisban hosszában ... % stenosist okozó meszes plaque."

# 1.4 Teljes példa egy feldolgozandó szöveg részre:
#
# Endarterectomia utáni állapot. Az arteria carotis communis 10-20 % stenosist okozó meszes plaque. A carotis oszlásban (bifurcatioban) 20-30 % stenosist okozó meszes plaque. Az arteria carotis interna elején 20 % stenosist okozó meszes plaque.
#
# Felismerendő minta: Az arteria carotis communis 10-20 % stenosist okozó meszes plaque.
# Kiolvasandó érték: 10-20

# 1.5 Architektúra:
#
# A feladatot megoldó neurális hálózat általánosítható több minta felépítésére.
#
# Klasszikus értelemben vett címkézett adatok nem állnak rendelkezésre. A hálózatnak magának kell megtanulnia mely szekvencia minta és mely nem az.
# 
# A címkézés hiányátt a következő trükkel oldhatjuk fel:
#
# Két minta akkor van közel egymáshoz ha bennük lévő szavak jelentése és sorrendisége megegyezik. A szemantikai egyezés úgy vizsgáltam, hogy a cél mintákban és tanuló adathalmazban lévő szavakat beágyazással vektorizáltam. Ezután két mondat szemantikai egyezőségét az fogja megadni, hogy a cosinus hasonlósága átlagosan milyen messze van a mondatokban azonos helyen lévő szavaknak.
#
# Például legyen a minták és a vizsgálandó szövegrész a következő:
# Minta: arteria carotis communis
# Szöveg: arteria carotis hosszában
# A minta és a szöveg első két szava megegyezik tehát ott a coszinus hasonlóság 1 még az utolsó szavak esetében valamilyen -1-nél nagyobb de 1 nél kisebb szám lesz. Ha ezeket összeadjuk és elosszuk a hosszal ami jelen példában 3 megkapjuk az átlagos hasonlóságát a két mondatnak. 
# A cosinus hasonlóság mindig -1 és 1 közötti szám ami nem előnyös ezért azt a következő képlettel 0 és 1 köz szorítom: (1 + cos_sim / 2)
#
# Ha ezzel a módszerrel előállítjuk a címkéket akkor a tanuló halmaz összes szekvenciájához (amit úgy állítunk elő hogy ugyanolyan hosszú darabokra tördeljük a forrás szöveget mint amilyen a minta hossza) rendelünk egy 0 és 1 közötti számot. Ez a szám megadja majd, hogy az adott szekvencia milyen közel van a mintánkhoz. 
#
# Az előállított adathalmazhoz egyszerűen építhető egy komplex háló ami akár 99.99% pontosággal rá tud tanulni a címkékre. A háló felépítésénél az fontos, hogy mindenképpen tartalmaznia kell egy Embedding layer-t a beágyazás miatt és egy LSTM háló a szavak szórendiségének figyelése miatt. Egy ’sima’ sűrűn kapcsolt hálónál ez a felépítés lényegesen jobban működött.
#
# Az aktivációs függvény sigmoid emiatt kell 0 és 1 közé szorítani a cosinus hasonlóságot. Veszteség függvénynek mean squared error. Metrika pedig a Root Mean Squared Error vagy Mean Absolute Error is megfelelő. 


In [52]:
# 2.0 Adatok előkszítése
import pandas

soruce = "sources/source_idydptc_0_10000.xlsx"
data = pandas.DataFrame()
xl = pandas.ExcelFile(soruce)
parse = xl.parse(xl.sheet_names)
sheet = xl.sheet_names[0]
data = parse[sheet]
# 2.1 Az adathalmaz első 10 sorra. Egy lehetséges minta a 3. és 4 sorokban található
data["obsval"][:10]

0                                             36&&1.21
1                                               90&&50
2                                               34&&21
3    Endarterectomia utáni állapot. Az arteria caro...
4    Az arteria carotis communis egész hosszában 10...
5                                   NORMÁLIS&&NORMÁLIS
6                                               67&&39
7                                               54&&27
8    A subclaviaban signifikáns szűkületre utaló ár...
9                                                _igen
Name: obsval, dtype: object

In [53]:
# 3.0 corpus elkészítes word2vec-hez
from keras_preprocessing import text

output_corp = open("sources/corpus.txt", "w", encoding="utf-8")

lenv = len(data["obsval"])
my_filters = '"#$&()*+/:;<=>?@[\\]^_`{|}~\t\n'
for i in range(0, lenv):
    tmp = str(data["obsval"][i]).lower()
    tmp = text.text_to_word_sequence(text=tmp,
                                        filters=my_filters)
    tmp = " ".join(map(str, tmp))
    output_corp.write(tmp + "\n")

output_corp.close()

# 3.1 A corpus első 200 karaktere
f = open("sources/corpus.txt", "r", encoding="utf-8")
print(f.read()[:200])
f.close()

36 1.21
90 50
34 21
endarterectomia utáni állapot. az arteria carotis communis egész hosszában 10-20 % stenosist okozó meszes plaque. a carotis oszlásban bifurcatioban 20-30 % stenosist okozó meszes p


In [54]:
# 4.0 Minták betöltése és tanuló adathalmaz előkészítése:

output_train = open("sources/corpus_train.txt", "w", encoding="utf-8")
# "next_tokens_count" - A mintában hány szó van a kiolvasandó érték után
next_tokent_count = 4
# "prev_tokens_count" - A mintában hány szó van a kiolvasandó érték előtt
prev_tokens_count = 5
# Például az alábbi mintában a kiolvasandó érték "az arteria carotis communisban" szavak után van és a "% stenosist okozó echogen plaque." szavak előtt.

# sample_count - Megadja, hogy a tanuló adathalmazt mennyi plusz mintával dusítjuk.
sample_count = 5000
# "sample" - A minta összes szava
sample = ["az arteria carotis communisban % stenosist okozó echogen plaque."]
for i in range(0, lenv):
    try:
        tmp_split = data["obsval"][i].lower().split(" ")
        len_tmp = len(tmp_split)
        for index in range(0, len_tmp):

            tmp_next_tokent_count = index + 1 + next_tokent_count
            tmp_prev_tokens_count = index - prev_tokens_count

            if tmp_prev_tokens_count >= 0 and tmp_next_tokent_count < len_tmp and len(tmp_split[index]) > 0:
                env = tmp_split[tmp_prev_tokens_count:index] + \
                    tmp_split[(index+1):tmp_next_tokent_count]

                output_train.write(tmp_split[index] +
                                "\t" + " ".join(env) + "\n")
    except:
        continue

for j in range(0, sample_count):
    output_train.write("SAMPLE\t" + sample[0] + "\n")

output_train.close()

# A tanuló adathalmaz első 500 karaktere
f = open("sources/corpus_train.txt", "r", encoding="utf-8")
print(f.read()[:500])
f.close()

carotis	endarterectomia utáni állapot. az arteria communis egész hosszában 10-20
communis	utáni állapot. az arteria carotis egész hosszában 10-20 %
egész	állapot. az arteria carotis communis hosszában 10-20 % stenosist
hosszában	az arteria carotis communis egész 10-20 % stenosist okozó
10-20	arteria carotis communis egész hosszában % stenosist okozó meszes
%	carotis communis egész hosszában 10-20 stenosist okozó meszes plaque.
stenosist	communis egész hosszában 10-20 % okozó meszes plaque. a
oko


In [55]:
# 5.0 Corpus beágyazása
import gensim # https://www.pydoc.io/pypi/gensim-3.2.0/autoapi/models/word2vec/index.html

embedding_dim = 16

model = gensim.models.Word2Vec(corpus_file="sources/corpus.txt",
                                window=10,
                                size=embedding_dim,
                                iter=10,
                                min_count=1)

model.save("word2vec.model")
model.wv.save_word2vec_format("word2vec_pattern.vec")

# 5.1 A beágyazott szókészlet mérete és első szavának vektora
f = open("word2vec_pattern.vec", "r", encoding="utf-8")
print(f.readline())
print(f.readline())
f.close()

1759 16

a 1.2400651 1.3711394 1.261693 0.8345974 0.2602962 -5.0107775 2.0614645 0.43603092 1.0035423 1.3180683 -3.2980626 0.30995876 -0.03683576 -0.19721483 0.81641406 -0.018020403



In [56]:
# 6.0 Tanuló adatok betöltése és Tanuló/Teszt felbontás, tokenizálás elvégzése
soruce = "sources/corpus_train.txt"
samples = ["az arteria carotis communisban % stenosist okozó echogen plaque."]
maxlen = max(len(x.split(" ")) for x in samples)

x_train = []
y_train = []

# 6.1 corpus beolvasása
f = open(soruce, "r", encoding="utf-8")
tmp_buff = f.read().split("\n")
lenb = len(tmp_buff)
for i in range(0, lenb):
    if(len(tmp_buff[i]) == 0):
        continue

    tmp_row = tmp_buff[i].split("\t")
    x_train.append(tmp_row[1])
f.close()

# 6.2 A tanuló adathalmaz első eleme
print(x_train[0])

endarterectomia utáni állapot. az arteria communis egész hosszában 10-20


In [57]:
# 7.0 Tokenizálás

import numpy
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, filters=my_filters)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
tmp_samples = tokenizer.texts_to_sequences(samples)
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
tmp_samples = pad_sequences(tmp_samples, padding='post', maxlen=maxlen)
y_train = numpy.copy(x_train).astype(numpy.int32)

tokenizer_json = tokenizer.to_json()
tokenizer_json = {
    "max_lenght": maxlen,
    "vocab_size": len(tokenizer.word_index) + 1,
    "tokens": tokenizer_json
}

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer,
                handle,
                protocol=pickle.HIGHEST_PROTOCOL)

# 7.1 Az adathalmaz első 5 eleme tokenizálás után
print(x_train[:4])
# 7.2 A minta tokenizálás után
print(tmp_samples)

[[ 91  79  77   2   7  35 140 151  25]
 [ 79  77   2   7   1 140 151  25   3]
 [ 77   2   7   1  35 151  25   3   4]
 [  2   7   1  35 140  25   3   4   6]]
[[ 2  7  1 18  3  4  6 21  8]]


In [58]:
# 8.0 Tanuló adatok felbontása teszt és tanló adathalmazra 80/20 arányban
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train,
                                                    y_train,
                                                    train_size=0.8,
                                                    test_size=0.2,
                                                    random_state=1000)

# Tanuló adathalmaz mérete
print(len(x_train))
# Teszt adathalmaz mérete
print(len(x_test))

56281
14071


In [59]:
# 9.0 Az előre elkészített beágyazás betöltése
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = numpy.zeros((vocab_size, embedding_dim))
embedding_matrix[0].fill(-1.0)

f = open("word2vec_pattern.vec", encoding='utf8', errors='ignore')
index = 0
for row in f:
    word, *vector = row.split()
    if word in tokenizer.word_index:
        idx = tokenizer.word_index[word]
        embedding_matrix[idx] = numpy.array(vector, dtype=numpy.float32)[:embedding_dim]

    index = index + 1

f.close()

# 9.1 Beágyazást táróló mátrix 2. eleme
print(embedding_matrix[1])

[ 4.18737221  1.95336926  4.75430059 -0.88585562 -3.26712275 -1.85779703
  0.34983012 -0.91726637 -0.45905134  3.02688694 -2.6880765   0.00664545
  2.87261868 -1.41956437  2.16651821  0.96636367]


In [60]:
# 10. Model összeállítása
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional

model = models.Sequential()
model.add(Embedding(len(tokenizer.index_word)+1,
                    output_dim=embedding_dim,
                    input_length=maxlen,
                    weights=[embedding_matrix],
                    trainable=True))
model.add(LSTM(embedding_dim))

model.add(Dense(len(samples), activation="sigmoid"))

model.compile(loss="mse",
                optimizer='adam',
                metrics=['mae', tf.keras.metrics.RootMeanSquaredError(name='rmse')])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 9, 16)             23728     
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 25,857
Trainable params: 25,857
Non-trainable params: 0
_________________________________________________________________


In [79]:
# 11. Cimkék előállítása.
from sklearn.metrics.pairwise import cosine_similarity

def conver_to_data(data, weights, samples, tokenizer):

    lenv = len(data)
    lens = len(samples[0])
    lenss = len(samples)

    weights_sim = cosine_similarity(weights)
    weights_sim = 1 + weights_sim
    weights_sim = weights_sim / 2
    weights_sim = weights_sim / lens
    similarities = []
    for i in range(0, lenv):
        calc_buffer = []
        for n in range(0, lenss):
            calc_buffer.append(0.0)

        for j in range(0, lens):
            for n in range(0, lenss):
                calc_buffer[n] = calc_buffer[n] + weights_sim[data[i][j]][samples[n][j]] 

        similarities.append(calc_buffer)

    return numpy.asarray(similarities)

y_train_sim = conver_to_data(y_train,
                            model.layers[0].get_weights()[0],
                            tmp_samples,
                            tokenizer)

y_test_sim = conver_to_data(y_test,
                            model.layers[0].get_weights()[0],
                            tmp_samples,
                            tokenizer)

# A tanuló cimkék első elem
print(y_train_sim[0])
# A teszt cimkék első eleme
print(y_test_sim[0])

[0.77545293]
[0.86206971]


In [82]:
# 12.0 Model betanítása
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('model_{epoch:03d}-{mae:03f}-{val_mae:03f}_pattern.h5',
                                 verbose=1,
                                 monitor='val_loss',
                                 save_best_only=True,
                                 mode='auto')

history = model.fit(x_train,
                y_train_sim,
                epochs=5,
                batch_size=100,
                validation_data=(x_test, y_test_sim),
                callbacks=[checkpoint],
                verbose=1)

# Tanulási történet kiíratása
print(history.history)

Train on 56281 samples, validate on 14071 samples
Epoch 1/5
56200/56281 [============================>.] - ETA: 0s - loss: 1.0969e-04 - mae: 0.0079 - rmse: 0.0105
Epoch 00001: val_loss improved from inf to 0.00011, saving model to model_001-0.007867-0.007672_pattern.h5
56281/56281 [==============================] - 6s 107us/sample - loss: 1.0964e-04 - mae: 0.0079 - rmse: 0.0105 - val_loss: 1.0647e-04 - val_mae: 0.0077 - val_rmse: 0.0103
Epoch 2/5
56000/56281 [============================>.] - ETA: 0s - loss: 8.9053e-05 - mae: 0.0071 - rmse: 0.0094
Epoch 00002: val_loss improved from 0.00011 to 0.00009, saving model to model_002-0.007104-0.006822_pattern.h5
56281/56281 [==============================] - 6s 108us/sample - loss: 8.9024e-05 - mae: 0.0071 - rmse: 0.0094 - val_loss: 8.6836e-05 - val_mae: 0.0068 - val_rmse: 0.0093
Epoch 3/5
56000/56281 [============================>.] - ETA: 0s - loss: 7.5187e-05 - mae: 0.0065 - rmse: 0.0087
Epoch 00003: val_loss improved from 0.00009 to 0.00

In [85]:
# 13.0 Model működésének tesztelése
model = models.load_model("model_005-0.008607-0.008416_pattern.h5", compile=False)

test_samples = ["az arteria carotis communisban % stenosist okozó echogen plaque.",
            "az arteria carotis communisban % stenosist okozó meszes plaque.",
            "arteria carotis communisban több % stenosist okozó meszes plaque.",
            "a arteria carotis interna elején % stenosist okozó echogen plaque.",
            "a arteria carotis interna elején % stenosist okozó meszes plaque.",
            "carotis oszlásban (bifurcatioban) % stenosist okozó echogen plaque.",
            "carotis oszlásban (bifurcatioban) % stenosist okozó meszes plaque.",
            "a carotis externában % stenosist okozó echogen plaque.",
            "a carotis externában % stenosist okozó meszes plaque.",
            "arteria vertebralisban az áramlási sebesség a norm. érték alsó határának %-a.",
            "arteria basilarisban az áramlási sebesség a norm. érték alsó határának %-a.",
            "endarterectomia utáni állapot. az carotis communis egész hosszában 10-20",
            "a. carotis externában áramlási sebesség alapján 50-60 %-os"]

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

test_samples = tokenizer.texts_to_sequences(test_samples)
test_samples = pad_sequences(test_samples,
                            padding='post',
                            maxlen=maxlen)

# A model predikciója a mintával való egyezéssel
# Látható, hogy az első minta egyezik meg a tanuló mintánkkal ezért ott a legerősebb a predikció. De vannak más minták is amik közel vannak hozzá. Feldolgozás során egy küszöbb értékkel tudjuk szabályozni, hogy mennyire szoros mintákat vizsgálunk.
print(model.predict(test_samples))

[[0.98857635]
 [0.97920734]
 [0.92161614]
 [0.94661504]
 [0.9095697 ]
 [0.88668615]
 [0.8746072 ]
 [0.8607577 ]
 [0.84764856]
 [0.626014  ]
 [0.626014  ]
 [0.76784456]
 [0.642442  ]]
